In [14]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import json

In [15]:
filename = "rewiring results final.csv"

In [16]:
def nclusters_meanshift(data, bw):
    ops = {i: data[i] for i in range(len(data))}
    sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
    A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
    clustering = MeanShift(bandwidth=bw).fit(A)
    lbls = clustering.labels_
    labels = np.arange(len(sorted_ops))
    for i in range(len(labels)):
        cl = lbls[i]
        labels[sorted_ops[i][0]]=cl
    cluster_participation_dict = {}
    for l in labels:
        if l not in cluster_participation_dict:
            cluster_participation_dict[l] = 1
        else:
            cluster_participation_dict[l] += 1
    #computing effective number of clusters using function explained in the paper
    C_num = 0
    C_den = 0
    for k in cluster_participation_dict:
        C_num += cluster_participation_dict[k]
        C_den += ((cluster_participation_dict[k])**2)
    C_num = (C_num**2)
    C = C_num/C_den
    return C

def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

In [17]:
results = pd.read_csv("aggregate/{}".format(filename), index_col=[0])
results.head()

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter
0,er,250,0.1,0.2,0.0,0.0,0.0,1000000,1.994783,0.201817,0.115975,0.204666,54.21,25.258780
1,er,250,0.1,0.2,0.2,0.0,0.0,1000000,2.245156,0.333598,0.117917,0.204700,109.71,113.012049
2,er,250,0.1,0.2,0.2,0.2,0.0,1000000,2.211670,0.328518,0.117024,0.203771,93.13,53.685326
3,er,250,0.1,0.2,0.4,0.0,0.0,1000000,2.336074,0.426069,0.118608,0.205062,86.24,26.657620
4,er,250,0.1,0.2,0.4,0.4,0.0,1000000,2.329744,0.378100,0.119049,0.206666,83.06,23.724351


In [18]:
new_values_avg = []
new_values_std = []

In [19]:
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    gr = row['gr']
    name = f'rewiring pr{pr} e{eps} g{gam} gr{gr} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc = nclusters_meanshift(finalops, bw=0.01)
        ncarray.append(nc)
    ncarray = np.array(ncarray)
    new_values_avg.append(np.average(ncarray))
    new_values_std.append(np.std(ncarray))

doing rewiring pr0.0 e0.2 g0.0 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.2 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.2 gr0.2 mi1000000
doing rewiring pr0.0 e0.2 g0.4 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.4 gr0.4 mi1000000
doing rewiring pr0.0 e0.2 g0.6 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.6 gr0.6 mi1000000
doing rewiring pr0.0 e0.2 g0.8 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.8 gr0.8 mi1000000
doing rewiring pr0.0 e0.2 g1.0 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.0 gr1.0 mi1000000
doing rewiring pr0.0 e0.2 g1.2 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.2 gr1.2 mi1000000
doing rewiring pr0.0 e0.2 g1.4 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.4 gr1.4 mi1000000
doing rewiring pr0.0 e0.2 g1.6 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.6 gr1.6 mi1000000
doing rewiring pr0.0 e0.3 g0.0 gr0.0 mi1000000
doing rewiring pr0.0 e0.3 g0.2 gr0.0 mi1000000
doing rewiring pr0.0 e0.3 g0.2 gr0.2 mi1000000
doing rewiring pr0.0 e0.3 g0.4 gr0.0 mi1000000
doing rewirin

In [20]:
results["0.01MS_avg_ncluster"] = new_values_avg
results["0.01MS_std_ncluster"] = new_values_std

In [21]:
new_values_avg = []
new_values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    gr = row['gr']
    name = f'rewiring pr{pr} e{eps} g{gam} gr{gr} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc = nclusters_meanshift(finalops, bw=0.00001)
        ncarray.append(nc)
    ncarray = np.array(ncarray)
    new_values_avg.append(np.average(ncarray))
    new_values_std.append(np.std(ncarray))

results["0.00001MS_avg_ncluster"] = new_values_avg
results["0.00001MS_std_ncluster"] = new_values_std

doing rewiring pr0.0 e0.2 g0.0 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.2 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.2 gr0.2 mi1000000
doing rewiring pr0.0 e0.2 g0.4 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.4 gr0.4 mi1000000
doing rewiring pr0.0 e0.2 g0.6 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.6 gr0.6 mi1000000
doing rewiring pr0.0 e0.2 g0.8 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g0.8 gr0.8 mi1000000
doing rewiring pr0.0 e0.2 g1.0 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.0 gr1.0 mi1000000
doing rewiring pr0.0 e0.2 g1.2 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.2 gr1.2 mi1000000
doing rewiring pr0.0 e0.2 g1.4 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.4 gr1.4 mi1000000
doing rewiring pr0.0 e0.2 g1.6 gr0.0 mi1000000
doing rewiring pr0.0 e0.2 g1.6 gr1.6 mi1000000
doing rewiring pr0.0 e0.3 g0.0 gr0.0 mi1000000
doing rewiring pr0.0 e0.3 g0.2 gr0.0 mi1000000
doing rewiring pr0.0 e0.3 g0.2 gr0.2 mi1000000
doing rewiring pr0.0 e0.3 g0.4 gr0.0 mi1000000
doing rewirin

In [22]:
results

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter,0.01MS_avg_ncluster,0.01MS_std_ncluster,0.00001MS_avg_ncluster,0.00001MS_std_ncluster
0,er,250,0.1,0.2,0.0,0.0,0.0,1000000,1.994783,0.201817,0.115975,0.204666,54.21,25.258780,1.994703,0.200727,3.143909,1.361087
1,er,250,0.1,0.2,0.2,0.0,0.0,1000000,2.245156,0.333598,0.117917,0.204700,109.71,113.012049,2.111855,0.245659,7.008035,1.789427
2,er,250,0.1,0.2,0.2,0.2,0.0,1000000,2.211670,0.328518,0.117024,0.203771,93.13,53.685326,2.080904,0.195895,6.806279,1.541695
3,er,250,0.1,0.2,0.4,0.0,0.0,1000000,2.336074,0.426069,0.118608,0.205062,86.24,26.657620,2.173097,0.321109,7.548713,2.482634
4,er,250,0.1,0.2,0.4,0.4,0.0,1000000,2.329744,0.378100,0.119049,0.206666,83.06,23.724351,2.134819,0.266441,7.630505,2.284513
5,er,250,0.1,0.2,0.6,0.0,0.0,1000000,2.430064,0.363504,0.118173,0.203946,86.90,23.349994,2.234452,0.264980,8.356494,2.142912
6,er,250,0.1,0.2,0.6,0.6,0.0,1000000,2.442384,0.403754,0.117621,0.202873,85.05,24.933599,2.242129,0.317459,8.164969,2.126945
7,er,250,0.1,0.2,0.8,0.0,0.0,1000000,2.750083,0.616820,0.119521,0.203637,94.00,32.994643,2.458553,0.443122,9.698474,2.933694
8,er,250,0.1,0.2,0.8,0.8,0.0,1000000,2.660078,0.521801,0.119146,0.204109,87.20,24.975139,2.419370,0.387732,9.470426,2.593060
9,er,250,0.1,0.2,1.0,0.0,0.0,1000000,3.252232,0.856886,0.124084,0.207283,106.39,50.159120,2.876466,0.639495,11.504293,4.105606


In [23]:
results.to_csv("aggregate/rewiring results final final.csv")

In [24]:
results = pd.read_csv("aggregate/rewiring results final final.csv", index_col=[0])
results

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter,0.01MS_avg_ncluster,0.01MS_std_ncluster,0.00001MS_avg_ncluster,0.00001MS_std_ncluster
0,er,250,0.1,0.2,0.0,0.0,0.0,1000000,1.994783,0.201817,0.115975,0.204666,54.21,25.258780,1.994703,0.200727,3.143909,1.361087
1,er,250,0.1,0.2,0.2,0.0,0.0,1000000,2.245156,0.333598,0.117917,0.204700,109.71,113.012049,2.111855,0.245659,7.008035,1.789427
2,er,250,0.1,0.2,0.2,0.2,0.0,1000000,2.211670,0.328518,0.117024,0.203771,93.13,53.685326,2.080904,0.195895,6.806279,1.541695
3,er,250,0.1,0.2,0.4,0.0,0.0,1000000,2.336074,0.426069,0.118608,0.205062,86.24,26.657620,2.173097,0.321109,7.548713,2.482634
4,er,250,0.1,0.2,0.4,0.4,0.0,1000000,2.329744,0.378100,0.119049,0.206666,83.06,23.724351,2.134819,0.266441,7.630505,2.284513
5,er,250,0.1,0.2,0.6,0.0,0.0,1000000,2.430064,0.363504,0.118173,0.203946,86.90,23.349994,2.234452,0.264980,8.356494,2.142912
6,er,250,0.1,0.2,0.6,0.6,0.0,1000000,2.442384,0.403754,0.117621,0.202873,85.05,24.933599,2.242129,0.317459,8.164969,2.126945
7,er,250,0.1,0.2,0.8,0.0,0.0,1000000,2.750083,0.616820,0.119521,0.203637,94.00,32.994643,2.458553,0.443122,9.698474,2.933694
8,er,250,0.1,0.2,0.8,0.8,0.0,1000000,2.660078,0.521801,0.119146,0.204109,87.20,24.975139,2.419370,0.387732,9.470426,2.593060
9,er,250,0.1,0.2,1.0,0.0,0.0,1000000,3.252232,0.856886,0.124084,0.207283,106.39,50.159120,2.876466,0.639495,11.504293,4.105606


In [25]:
results

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter,0.01MS_avg_ncluster,0.01MS_std_ncluster,0.00001MS_avg_ncluster,0.00001MS_std_ncluster
0,er,250,0.1,0.2,0.0,0.0,0.0,1000000,1.994783,0.201817,0.115975,0.204666,54.21,25.258780,1.994703,0.200727,3.143909,1.361087
1,er,250,0.1,0.2,0.2,0.0,0.0,1000000,2.245156,0.333598,0.117917,0.204700,109.71,113.012049,2.111855,0.245659,7.008035,1.789427
2,er,250,0.1,0.2,0.2,0.2,0.0,1000000,2.211670,0.328518,0.117024,0.203771,93.13,53.685326,2.080904,0.195895,6.806279,1.541695
3,er,250,0.1,0.2,0.4,0.0,0.0,1000000,2.336074,0.426069,0.118608,0.205062,86.24,26.657620,2.173097,0.321109,7.548713,2.482634
4,er,250,0.1,0.2,0.4,0.4,0.0,1000000,2.329744,0.378100,0.119049,0.206666,83.06,23.724351,2.134819,0.266441,7.630505,2.284513
5,er,250,0.1,0.2,0.6,0.0,0.0,1000000,2.430064,0.363504,0.118173,0.203946,86.90,23.349994,2.234452,0.264980,8.356494,2.142912
6,er,250,0.1,0.2,0.6,0.6,0.0,1000000,2.442384,0.403754,0.117621,0.202873,85.05,24.933599,2.242129,0.317459,8.164969,2.126945
7,er,250,0.1,0.2,0.8,0.0,0.0,1000000,2.750083,0.616820,0.119521,0.203637,94.00,32.994643,2.458553,0.443122,9.698474,2.933694
8,er,250,0.1,0.2,0.8,0.8,0.0,1000000,2.660078,0.521801,0.119146,0.204109,87.20,24.975139,2.419370,0.387732,9.470426,2.593060
9,er,250,0.1,0.2,1.0,0.0,0.0,1000000,3.252232,0.856886,0.124084,0.207283,106.39,50.159120,2.876466,0.639495,11.504293,4.105606
